In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import torch
import pandas as pd
from torchmetrics.text import ROUGEScore
from transformers import BartForConditionalGeneration, BartTokenizerFast
from tqdm.notebook import tqdm
import random

In [3]:
df = pd.read_csv("../../../data/labelled/reviews/test.csv")

In [4]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device

'cuda'

In [5]:
model_path = "facebook/bart-large-cnn"

In [6]:
model = BartForConditionalGeneration.from_pretrained(model_path)

In [7]:
%%capture
model.to(device)

In [8]:
tokenizer = BartTokenizerFast.from_pretrained(model_path)

In [9]:
rouge = ROUGEScore()

In [10]:
batch_size = 4

In [11]:
df.head(2)

,input_reviews,label_reviews
0,Below are the reviews of a product: \n- I purc...,pros:\n- Reduces glare\n- Very impressive\n- W...
1,Below are the reviews of a product: \n- Well w...,pros:\n- Huge capacity\n- Beautiful design\n- ...


In [12]:
rouge_scores = {"rouge1":[], "rouge2":[], "rougeL": []}
preds_all = []
for idx in tqdm(range(0, len(df), batch_size)):
    batch = df[idx:idx+batch_size]
    product_descriptions = batch.input_reviews.to_list()
    actual_summary = batch.label_reviews.to_list()

    input_ids = tokenizer(product_descriptions, max_length=1024, return_tensors="pt", padding=True, truncation=True)["input_ids"].to(device)
    summary_ids = model.generate(input_ids, min_length=0, max_length=1024)
    preds = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    preds_all.extend(preds)

    for pred, target in zip(preds, actual_summary):
        rouge_all = rouge(pred, target)
        rouge_scores['rouge1'].append(rouge_all['rouge1_fmeasure'].item())
        rouge_scores['rouge2'].append(rouge_all['rouge2_fmeasure'].item())
        rouge_scores['rougeL'].append(rouge_all['rougeL_fmeasure'].item())


  0%|          | 0/5000 [00:00<?, ?it/s]

c:\Users\Engineer_AI6\miniconda3\envs\test_infra\lib\site-packages\transformers\models\bart\modeling_bart.py:590: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [13]:
df['rouge1'] = rouge_scores['rouge1']
df['rouge2'] = rouge_scores['rouge2']
df['rougeL'] = rouge_scores['rougeL']
df['bart_cnn'] = preds_all

In [14]:
df.to_csv("bart_cnn_scores.csv", index=False)

In [15]:
df = pd.read_csv("bart_cnn_scores.csv")

In [16]:
df.head()

,input_reviews,label_reviews,rouge1,rouge2,rougeL,bart_cnn
0,Below are the reviews of a product: \n- I purc...,pros:\n- Reduces glare\n- Very impressive\n- W...,0.262069,0.083916,0.151724,Sigma EX DG ultraviolet filter is hands-down t...
1,Below are the reviews of a product: \n- Well w...,pros:\n- Huge capacity\n- Beautiful design\n- ...,0.122137,0.015504,0.061069,"Anker Astro Pro is a rechargeable 5,000Ah batt..."
2,Below are the reviews of a product: \n- Charge...,pros:\n- Charges fast\n- Works well\n- Charges...,0.266667,0.027397,0.186667,RAVPower is all about quality and reliability...
3,Below are the reviews of a product: \n- Given ...,pros:\n- Very rugged and shock resistant\n- Pr...,0.276730,0.089172,0.150943,Reviewer: This is the best case available for ...
4,Below are the reviews of a product: \n- I orde...,pros:\n- Sturdy packaging and build (stand up ...,0.256410,0.031088,0.112821,The Powerchute CD is a cord-driven UPS that ca...


In [17]:
scores = {"rouge1": df.rouge1.mean(), "rouge2": df.rouge2.mean(), "rougeL": df.rougeL.mean()}

In [18]:
scores

{'rouge1': 0.23264151072856037,
 'rouge2': 0.0755656192275346,
 'rougeL': 0.15192922783442772}